# Algoritmos Heurísticos e Mta-Heurísticas para Problemas de Otimização

Implementação de heurísticas para resolução de problema de escalonamento.

# Problema de Escalonamento em Máquinas Paralelas (PEMP)

Problemas onde há uma quantidade de tarefas a serem processadas pelas máquinas. Cada tarefa precisa ser processada uma única vez, e pode ser processada em qualquer máquina.

![pmsp](./figures_2/pmsp.png)

## Representação da solução

Para garantir o sucesso e facilitar a implementação de uma heurística, a escolha da representação da solução desempenha um papel fundamental. No contexto do Problema de Escalonamento em Máquinas Paralelas (PEMP) abordado aqui, a solução pode ser adequadamente representada por meio de um vetor. Nesse vetor, cada posição representa uma tarefa, e o valor contido em cada posição indica a máquina sugerida para alocar essa tarefa. 

Por exemplo:

\begin{bmatrix}
    1 & 0 & 0 & 1 & 0
\end{bmatrix}

Representa a solução 

![](./figures_2/pmsp_solution.png)

<font color='red'> OBS: Para o presente problema, não faz diferença a ordem de processamento das tarefas, apenas em qual máquina elas estão alocadas. </font>

## Geração de instância aleatória

Desenvolva um algoritmo capaz de gerar uma matriz aleatória que represente a duração do processamento $p_{ij}$ de cada uma das $i$ tarefas em relação a cada uma das $j$ máquinas, com base nas quantidades especificadas de tarefas e máquinas.

Para facilitar a compreensão, consideremos o exemplo da matriz abaixo, onde $p_{32}$ representa o tempo de processamento da tarefa $3$ na máquina $2$ e tem o valor de $20$ unidades de tempo:

\begin{align}
\begin{bmatrix}
8 & 10 \\
9 & 11 \\
5 & 20
\end{bmatrix}
\end{align}

Você pode utilizar a função `np.random.randint` para criar essa matriz, sendo o intervalo utilizado $[5, 20)$ para gerar a duração das tarefas.

In [1]:
import numpy as np
import random

num_tarefas = 5
num_maquinas = 3

duracao_minima = 5
duracao_maxima = 20

np.random.seed(310)
duracao_processamento = np.random.randint(duracao_minima, duracao_maxima, (num_tarefas, num_maquinas))

## Criação de Instância 

Crie uma classe que armazene os dados da instância para ser utilizado em funções.

In [2]:
class Instancia:
    def __init__(self, num_tarefas, num_maquinas, processamento):
        self.num_tarefas = num_tarefas
        self.num_maquinas = num_maquinas
        self.processamento = processamento

instancia_1 = Instancia(num_tarefas, num_maquinas, duracao_processamento)

## Funções para soluções

Declare funções que:
- Gere solução vazia `gerar_solucao_vazia(instancia)`
- Gere solução aleatória `gerar_solucao_aleatoria(instancia)`
- Copie uma solução para outra já declarada `copiar_solucao(solucao_destino, solucao_origem)`

In [3]:
# Solução vazia
def gerar_solucao_vazia(instancia):
    return np.zeros(instancia.num_tarefas, int) 

In [4]:
# Solução aleatória
def gerar_solucao_aleatoria(instancia):
    return np.random.randint(0, instancia.num_maquinas, instancia.num_tarefas)

In [5]:
# Copiar solução
def copiar_solucao(solucao_destino, solucao_original):
    for (i, job) in enumerate(solucao_original):
        solucao_destino[i] = job
    return solucao_destino

In [6]:
# Teste
np.random.seed(1)
s1 = gerar_solucao_aleatoria(instancia_1)
s1

array([1, 0, 0, 1, 1])

## Objetivo: minimizar o somatório das durações

Implemente um código para calcular a função objetivo. Sugestão: utilize as funções `enumerate` e `sum`.

<!-- `enumerate` `sum` -->

In [7]:
for (i, j) in enumerate(s1):
    print(f'tarefa={i}, maquina={j}')

tarefa=0, maquina=1
tarefa=1, maquina=0
tarefa=2, maquina=0
tarefa=3, maquina=1
tarefa=4, maquina=1


In [8]:
def calcular_tempo_total(solucao, instancia):
    resultado = sum(instancia.processamento[i, j] for (i, j) in enumerate(solucao))
    return resultado

In [9]:
calcular_tempo_total(s1, instancia_1)

68

## 1. Simulated Annealing (SA)

### Parâmetros

O algoritmo SA envolve a consideração de parâmetros significativos para seu funcionamento. Dentre eles:
- $T_0$: temperatura inicial;
- $T_l$: limite inferior de temperatura;
- $L$: quantidade máxima de iterações;
- $f(T)=c \, T$: função que representa a taxa de diminuição da temperatura a cada iteração. 

Implemente uma classe que contenha esses dados. OBS: armazene apenas `c` no último parâmetro.

In [10]:
class ParametrosSA:
    def __init__(self, T_init, Tl, L, c):
        self.temperatura_inicial = T_init
        self.temperatura_inferior = Tl
        self.max_iteracoes = L
        self.taxa_diminuicao = c

### Estruturas de Vizinhança

Existem várias abordagens para realizar movimentos entre soluções a fim de encontrar soluções melhores. Para ilustração da estrutura de vizinhaça adotada, considere a seguinte solução, onde temos 2 máquinas e 5 tarefas.

<!-- \begin{bmatrix}
1 & 0 & 1 & 0 & 0
\end{bmatrix} -->

![vizinho_1](./figures_2/vizinhanca_1.png)

Uma solução vizinha poderia ser qualquer solução obtida através da realocação de qualquer tarefa para outra máquina, como:

<!-- \begin{bmatrix}
1 & 0 & 1 & 0 & 1
\end{bmatrix} -->

![vizinho_2](./figures_2/vizinhanca_2.png)

Desenvolva uma estrutura de vizinhança que seja capaz de escolher, de forma aleatória, uma solução vizinha a partir da solução original. Nessa estrutura, a operação fundamental envolve a troca da máquina à qual uma tarefa está alocada por outra máquina. 

In [11]:
# Troca por vizinhos aleatórios
def escolher_vizinho_aleatorio(solucao_copia, solucao_original, instancia):
    copiar_solucao(solucao_copia, solucao_original)
    realizar_troca_aleatoria(solucao_copia, instancia)
    return solucao_copia

In [12]:
def realizar_troca_aleatoria(solucao, instancia):
    tarefa_escolhida = np.random.randint(instancia.num_tarefas)
    maquina_escolhida = solucao[tarefa_escolhida]
    while solucao[tarefa_escolhida] == maquina_escolhida:
        maquina_escolhida = np.random.randint(instancia.num_maquinas)
    solucao[tarefa_escolhida] = maquina_escolhida
    return solucao

### Implementação
Implemente o SA se baseando no pseudocódigo abaixo:

![sa](./figures_2/sa.png)

In [13]:
import random
import math

def simulated_annealing(instancia, parametros, f = calcular_tempo_total):    
    T = parametros.temperatura_inicial
    
    solucao_atual = gerar_solucao_aleatoria(instancia)
    solucao_melhor = solucao_atual.copy()
    solucao_auxiliar = gerar_solucao_vazia(instancia)
    while T > parametros.temperatura_inferior:
        for k in range(parametros.max_iteracoes):
            escolher_vizinho_aleatorio(solucao_auxiliar, solucao_atual, instancia)            
            sentido = f(solucao_auxiliar, instancia) - f(solucao_atual, instancia)
            if sentido <= 0:
                copiar_solucao(solucao_atual, solucao_auxiliar)                
                if f(solucao_atual, instancia) < f(solucao_melhor, instancia):
                    copiar_solucao(solucao_melhor, solucao_atual)
                    
            elif random.random() < math.exp(-sentido/T):
                copiar_solucao(solucao_atual, solucao_auxiliar)
        T *= parametros.taxa_diminuicao
    return solucao_melhor, f(solucao_melhor, instancia)

In [14]:
parametros = ParametrosSA(100, 20, 40, 0.9)
simulated_annealing(instancia_1, parametros)

(array([0, 1, 0, 0, 2]), 39)

## 2. Algoritmos Genéticos (AG)

## Parâmetros

Assim como o SA, os AG dependem de diversos parâmetros para sua execução. Armazene em uma classe os seguintes parâmetros:
- Tamanho da população;
- Taxa de mutação (valor entre 0 e 1);
- Número de gerações.

In [15]:
class ParametrosGA:
    def __init__(self, tamanho_populacao, num_geracoes, taxa_mutacao=0.01):
        self.tamanho_populacao = tamanho_populacao
        self.taxa_mutacao = taxa_mutacao
        self.num_geracoes = num_geracoes

In [16]:
parametros_ga = ParametrosGA(10, 100, taxa_mutacao=0.1)

### População aleatória

Crie uma função que, fazendo uso da função já declarada para gerar soluções aleatórias, seja responsável por gerar uma população de soluções com base nos parâmetros previamente definidos. Uma sugestão seria nomear essa função como `gerar_populacao_aleatoria(instancia, parametros)`.

In [17]:
def gerar_populacao_aleatoria(instancia, parametros):
    return [gerar_solucao_aleatoria(instancia) for i in range(parametros.tamanho_populacao)]

### Seleção por Roleta

A seleção adotada para o presente minicurso é a seleção por roleta. Implemente uma solução que, ao receber uma população e sua adaptabilidade, seja capaz de selecionar e retornar uma solução com base na probabilidade estabelecida pela fórmula abaixo: 

$$P(s_i) = 1 - \frac{f(s_i)}{\sum_{j=1}^{N} f(s_j)}$$

Sugestão: antes de prosseguir com a implementação da seleção por roleta, é aconselhável criar uma função que calcule a adaptabilidade de toda a população.

In [18]:
def avaliar_adaptabilidade_populacao(populacao, instancia):
    return np.array([calcular_tempo_total(solucao, instancia) for solucao in populacao])

In [19]:
def selecionar_por_roleta(populacao, adaptabilidade):
    total = sum(adaptabilidade)
    probabilidade = [(1 - obj_i / total) for obj_i in adaptabilidade]
    return random.choices(populacao, probabilidade)[0]

### Cruzamento em Um Ponto

Implemente o cruzamento em um ponto `cruzar(filho, pai_1, pai_2)`. Exemplo:

![cruzamento](./figures_2/cruzamento.png)

<!-- Seja o Pai 1 representado pelo vetor abaixo:
\begin{bmatrix}
    1 & 0 & 0 & 0 & 0
\end{bmatrix}

E o Pai 2, pelo vetor:
\begin{bmatrix}
    0 & 1 & 0 & 1 & 1
\end{bmatrix}

Se o ponto selecionado aleatoriamente for 3, então o filho será:
\begin{bmatrix}
    1 & 0 & 0 & 1 & 1
\end{bmatrix} -->

In [20]:
def cruzar(filho, pai_1, pai_2):
    num_tarefas = len(pai_1)
    posicao = np.random.randint(num_tarefas)
    for i in range(num_tarefas):
        filho[i] = pai_1[i] if i < posicao else pai_2[i]
    return filho

### Mutação

Implemente a mutação que que realiza trocas aleatórias, como na estrutura de vizinhança do SA, com a frequência controlada pelo parâmetro previamente definido.

In [21]:
def mutar(filho, instancia, parametros):
    if random.random() < parametros.taxa_mutacao:
        realizar_troca_aleatoria(filho, instancia)
    return filho

### Algoritmo

In [22]:
def genetic_algorithm(instancia, parametros):
    populacao = gerar_populacao_aleatoria(instancia, parametros)
    adaptabilidade = avaliar_adaptabilidade_populacao(populacao, instancia)
    
    posicao_do_melhor = np.argmin(adaptabilidade)
    melhor_solucao = populacao[posicao_do_melhor]
    melhor_desempenho = adaptabilidade[posicao_do_melhor]
    
    for i in range(parametros.num_geracoes):
        
        nova_populacao = []
        for _ in range(parametros.tamanho_populacao):
            pai_1 = selecionar_por_roleta(populacao, adaptabilidade)
            pai_2 = selecionar_por_roleta(populacao, adaptabilidade)
            filho = gerar_solucao_vazia(instancia)
            cruzar(filho, pai_1, pai_2)
            mutar(filho, instancia, parametros)
            nova_populacao.append(filho)
        populacao = nova_populacao
        adaptabilidade = avaliar_adaptabilidade_populacao(populacao, instancia)
        
        posicao_do_melhor_gen_i = np.argmin(adaptabilidade)
        melhor_solucao_gen_i = populacao[posicao_do_melhor]
        melhor_desempenho_gen_i = adaptabilidade[posicao_do_melhor]

        if melhor_desempenho_gen_i < melhor_desempenho:
            melhor_solucao = melhor_solucao_gen_i
            melhor_desempenho = melhor_desempenho_gen_i
    return melhor_solucao, melhor_desempenho 
        

In [23]:
genetic_algorithm(instancia_1, parametros_ga)

(array([0, 1, 0, 0, 0]), 40)